# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
# загрузка библиотек
import pandas as pd

In [2]:
# чтение файла с данными и сохранение в data
data = pd.read_csv('c:\\Users\\79853\\Documents\\datascience\\yandex\\Спринт3\\data.csv')

# получение общей информации о данных в таблице data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
# получение первых 5 строк таблицы data
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


В таблице 12 столбцов. тип данных - int64 (5 столбцов), float64 (2 столбца), object (5 столбцов). Присутствуют количественные и качественные переменные. Количество значений в столбцах различается, значит, в данных есть пропущенные значения.

* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

In [4]:
# минимальные и максимальные значения количественных переменных
print('количество детей в семье', (data.children.min(), data.children.max()))
print('общий трудовой стаж в днях', (int(data.days_employed.min()), int(data.days_employed.max())))
print('возраст клиента в годах', (data.dob_years.min(), data.dob_years.max()))
print('ежемесячный доход', (data.total_income.min(), data.total_income.max()))

количество детей в семье (-1, 20)
общий трудовой стаж в днях (-18388, 401755)
возраст клиента в годах (0, 75)
ежемесячный доход (20667.26379327158, 2265604.028722744)


In [5]:
# уникальные значения в поле children
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

In [6]:
# подсчет количества строк со значением -1 в поле children
data[data.children == -1]['children'].count()

47

In [7]:
# подсчет количества строк со значением 20 в поле children
data[data.children == 20]['children'].count()

76

In [8]:
# Уникальные значения качественной переменной education
data.education.unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [9]:
# Уникальные значения качественной переменной education_id
data.education_id.unique()

array([0, 1, 2, 3, 4], dtype=int64)

In [10]:
# Уникальные значения качественной переменной family_status
data.family_status.unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [11]:
# Уникальные значения качественной переменной family_status_id
data.family_status_id.unique()

array([0, 1, 2, 3, 4], dtype=int64)

In [12]:
# Уникальные значения качественной переменной gender
data.gender.unique()

array(['F', 'M', 'XNA'], dtype=object)

In [13]:
# Уникальные значения качественной переменной income_type
data.income_type.unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [14]:
# Уникальные значения качественной переменной debt
data.debt.unique()

array([0, 1], dtype=int64)

In [15]:
# Уникальные значения качественной переменной purpose, отсортированные по алфавиту
sorted(data.purpose.unique())

['автомобили',
 'автомобиль',
 'высшее образование',
 'дополнительное образование',
 'жилье',
 'заняться высшим образованием',
 'заняться образованием',
 'на покупку автомобиля',
 'на покупку подержанного автомобиля',
 'на покупку своего автомобиля',
 'на проведение свадьбы',
 'недвижимость',
 'образование',
 'операции с жильем',
 'операции с коммерческой недвижимостью',
 'операции с недвижимостью',
 'операции со своей недвижимостью',
 'покупка жилой недвижимости',
 'покупка жилья',
 'покупка жилья для сдачи',
 'покупка жилья для семьи',
 'покупка коммерческой недвижимости',
 'покупка недвижимости',
 'покупка своего жилья',
 'получение высшего образования',
 'получение дополнительного образования',
 'получение образования',
 'приобретение автомобиля',
 'профильное образование',
 'ремонт жилью',
 'свадьба',
 'свой автомобиль',
 'сделка с автомобилем',
 'сделка с подержанным автомобилем',
 'строительство жилой недвижимости',
 'строительство недвижимости',
 'строительство собственной недв

In [16]:
# посмотрим нулевые значения в поле days_employed
data[data.days_employed.isnull()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


**Вывод**

В данных присутствуют количественные и категориальные переменные.

**Количественные:**
* `children`
* `days_employed`
* `dob_years`
* `total_income`

**Категориальные:**
* `education`
* `education_id`
* `family_status`
* `family_status_id`
* `gender`
* `income_type`
* `debt`
* `purpose`

1) В переменной `children` *(количество детей в семье)* два значения являются ошибочными:
- **-1** (47 значений) - переменная не может принимать отрицательное значение. данное значение заменим на **медианное значение** количества детей в семье.
- **20** (76 значений) - аномально большое значение для данной переменной (есть информация только про одну такую семью в России - семья Шишкиных из Воронежской области). Данные значения являются ошибочными, скорее всего, правильное значение **2**.

2) В переменной `dob_years` *(возраст клиента в годах)* присутствует некорректное значение **0**, т.к. минимальный возраст клиента, который может взять кредит составляет 18 лет. при обработке данных заменим эти значения на **средний возраст клиентов** в группах по видам занятости.

3) В переменной `days_employed` *(общий трудовой стаж в днях)*:
- отрицательные значения составляют 74% от общего количества данных. при обработке возьмем эти значения **по модулю**.
- максимальное значение поля 401755 (дней),что при пересчете в года составляет 1100. Данное значение является аномально большим. необходимо проанализировать все значения поля и выявить причину появления некорректных значений.

4) В качественных переменных `purpose` *(цель получения кредита)* и `education` *(уровень образования клиента)* присутствуют **неявные дубликаты**, которые вызваны в первом случае использованием разных формулировок для описания одной и той же цели и во втором случае - использованием символов в верхнем и нижнем регистре.

5) В переменной `gender` *(пол клиента)* присутствует неизвестное значение пола **'XNA'**.

6) Присутствуют пропуски данных в полях `days_employed` и `total_income`, которые совпадают по количеству - у клиентов сразу незаполнено оба поля. Вероятно, что датасет собирался из разных таблиц со сведениями о клиентах, в одной из которых отсутстввовали поля `days_employed` и `total_income`. 

7) Предварительно можно утверждать, что, данных достаточно для проверки гипотез.

## Шаг 2. Предобработка данных

### Обработка пропусков и некорректных значений

In [17]:
# замена некорректного значения -1 в поле children (количество детей в семье) медианным значением количества детей,
# преобразованным в целое число
data.loc[data.children == -1, 'children'] = int(data.children.median())

In [18]:
# замена некорректного значения 20 в поле children (количество детей в семье) значением 2
data.loc[data.children == 20, 'children'] = 2

In [19]:
# проверка уникальных значений в поле children после замены
data.children.unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

In [20]:
# Функция для замены некорректного значения 0 в поле dob_years (возраст клиента в годах) средним значением возраста клиентов
# в группах по типу занятости
# если соответствующего типа занятости нет в датасете, функция выводит среднее значение dob_years по всему датасету

def dob_years_pass(income_type):
    income_types = data['income_type'].unique()
    mean_dob_years = data.groupby('income_type')['dob_years'].mean() # среднее значение возраста для клиентов по типу занятости
    try:
        for element in income_types:
            if income_type == element:
                return int(mean_dob_years[element])
    except:
        return int(data.dob_years.mean())

In [21]:
# заполнение нулей в поле dob_years
data.loc[data.dob_years == 0, 'dob_years'] = data['income_type'].transform(lambda x: dob_years_pass(x))

In [22]:
# проверка уникальных значений в поле dob_years
sorted(data.dob_years.unique())

[19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75]

In [23]:
# подсчет пропущенных значений
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Исследуем поле `days_employed` (общий трудовой стаж в днях), добавим в датасет поле `years_employed` (общий трудовой стаж в годах) для рассчета отрицательные значения `days_employed` возьмем по модулю.

In [24]:
data['years_employed'] = abs(data['days_employed']) / 365

Исходя из максимального возраста клиента 75 лет, стаж работы не должен превышать 60 лет. Посчитаем количество значений в датафрейме со значением `years_employed` (общий трудовой стаж в годах) > 60 лет, выведем минимальное и максимальное значение `years_employed` (общий трудовой стаж в годах), чтобы убедиться в корректности фильтрации.

In [25]:
print('количество значений > 60 лет:', data[data.years_employed > 60]['years_employed'].count())
print('минимальное значение:', int(data[data.years_employed > 60]['years_employed'].min()), 'лет')
print('максимальное значение:', int(data[data.years_employed > 60]['years_employed'].max()), 'лет')

количество значений > 60 лет: 3445
минимальное значение: 900 лет
максимальное значение: 1100 лет


3445 значений в поле `days_employed` *(общий трудовой стаж в днях)* даны в другой единице измерения. вероятно, в часах.

Переведем аномально большие значений `days_employed` (общий трудовой стаж в днях) из часов в дни и пересчитаем все значения по модулю, чтобы убрать отрицательные значения.

In [26]:
data.loc[data.years_employed > 60, 'days_employed'] = data.days_employed / 24
data.days_employed = abs(data.days_employed)

In [27]:
# Функция для заполнения пропусков в поле days_employed (общий трудовой стаж в днях) средним значением стажа для
# клиентов такого же возраста
# если соответствующего возраста нет в датасете, функция выводит среднее значение days_employed по всему датасету

def days_employed_pass(dob_years):
    mean_days_employed = data.groupby('dob_years')['days_employed'].mean() # среднее значение стажа для клиентов по возрастам
    try:
        for age in range(18, 76):
            if dob_years == age:
                return mean_days_employed[age]
    except:
        return data.days_employed.mean() 

In [28]:
# заполнение пропусков в поле days_employed
data.loc[data.days_employed.isnull(), 'days_employed'] = data['dob_years'].transform(lambda x: days_employed_pass(x))

In [29]:
# пересчитаем поле years_employed (общий трудовой стаж в годах)
data['years_employed'] = data['days_employed'] / 365

In [30]:
# проверка минимальных и максимальных значений days_employed и years_employed
print('минимальное значение days_employed:', int(data['days_employed'].min()), 'дня')
print('минимальное значение years_employed:', int(data['years_employed'].min()), 'лет')
print()
print('максимальное значение days_employed:', int(data['days_employed'].max()), 'дней')
print('максимальное значение years_employed:', int(data['years_employed'].max()), 'лет')

минимальное значение days_employed: 24 дня
минимальное значение years_employed: 0 лет

максимальное значение days_employed: 18388 дней
максимальное значение years_employed: 50 лет


In [31]:
# Функция для заполнение пропусков в поле total_income медианным значением для каждого типа занятости
# если соответствующего типа занятости нет в датасете, функция выводит медианное значение total_income по всему датасету

def total_income_pass(income_type):
    income_types = data['income_type'].unique()
    median_total_income = data.groupby('income_type')['total_income'].median() # Рассчет медианного значения ежемесячного дохода (total_income) для каждого типа занятости
    try:
        for element in income_types:
            if income_type == element:
                return median_total_income[element]
    except:
        return data.total_income.median() 

In [32]:
# Заполнение пропусков в поле total_income
data.loc[data.total_income.isnull(), 'total_income'] = data['income_type'].transform(lambda x: total_income_pass(x))

In [33]:
# подсчет пропущенных значений
data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
years_employed      0
dtype: int64

**Вывод**

В поле `children` значения -1 заменены медианным значением переменной `children` для всего датасета, преобразованным в целое число, значения 20 - значением 2.

В поле `dob_years` значения 0 заменены средним значением переменной `dob_years` для групп по типу занятости `income_type`.

В поле `days_employed` отрицательные значения взяты по модулю, аномально большие значения преобразованы из часов в дни. для проверки корректности значений в поле `days_employed` в датасет добавлено поле `years_employed` (общий трудовой стаж в годах).

В поле `days_employed` пропуски заполнены средним значением переменной `days_employed` для клиентов такого же возраста `dob_years`.

### Замена типа данных

In [34]:
# замена типа данных на int в полях days_employed, years_employed и total_income
data.days_employed = data.days_employed.astype('int')
data.years_employed = data.years_employed.astype('int')
data.total_income = data.total_income.astype('int')

In [36]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int32 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int32 
 11  purpose           21525 non-null  object
 12  years_employed    21525 non-null  int32 
dtypes: int32(3), int64(5), object(5)
memory usage: 1.9+ MB


**Вывод**

Изменен тип данных в полях `days_employed`(общий трудовой стаж в днях), `years_employed` (общий трудовой стаж в годах), `total_income` (ежемесячный доход) с **float64** на **int64**.

### Обработка дубликатов

In [37]:
# подсчёт явных дубликатов
data.duplicated().sum()

54

In [38]:
# удаление явных дубликатов (с удалением старых индексов и формированием новых)
data = data.drop_duplicates().reset_index(drop=True)

In [39]:
# проверка на отсутствие дубликатов
data.duplicated().sum()

0

В поле `education` присутствуют неявные дубликаты, которые появились из-за использования символов в верхнем и нижнем регистре. Чтобы избавиться от дубликатов, приведем все символы к нижнему регистру.

In [40]:
data.education = data.education.str.lower()

In [41]:
# проверка результата по полю education
data.education.unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

**Вывод**

Устранены явные дубликаты.

Устранены неявные дубликаты в поле `education`

### Лемматизация

In [42]:
# загрузка библиотеки для лемматизации (приведение слова к его словарной форме)
from pymystem3 import Mystem
m = Mystem() 
pd.set_option('chained_assignment', None)

In [43]:
# функция для лемматизации текста
def lemma(series):
    '''
    Библиотека pymystem3 очень долго обрабатывает большое количество коротких текстов, однако довольно быстро
    обрабатывает длинные тексты, поэтому функция сначала объединит все короткие тексты в один текст с разделителем "|",
    лемматизирует его, и затем разобьет обратно на короткие лемматизированные тексты
    '''
    text = ''
    for i in range(len(series)):
        text = text + series[i] + '|'
    
    lemma = ''.join(m.lemmatize(text))
    lemma_list = lemma.split('|')
    
    for i in range(len(series)):
        series[i] = lemma_list[i]
    
    return series

In [44]:
%%time
data['purpose'] = lemma(data['purpose'])

Wall time: 6.97 s


In [45]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилье,23
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиль,11
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилье,15
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительный образование,11
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьба,38


Процесс лемматизации привел слова в поле `purpose` к словарной форме (лемме). теперь достаточно сформировать перечень основных целей и с помощью лемм заменить разные формулировки одной и той же цели одним вариантом.

Сгруппируем цели и определим для каждой группы лемму и значение, на которое будет произведена заменена.

| Номер группы | Цели из исходного датасета | Лемма | На что заменяем |
| :-: | :- | :- | :- |
| 1 | 'высшее образование', 'дополнительное образование', 'заняться высшим образованием', 'заняться образованием', 'образование', 'получение высшего образования', 'получение дополнительного образования', 'получение образования', 'профильное образование' | 'образование' | 'образование' |
|  |  |  |  |
| 2 | 'автомобили', 'автомобиль', 'на покупку автомобиля', 'на покупку подержанного автомобиля', 'на покупку своего автомобиля', 'приобретение автомобиля', 'свой автомобиль', 'сделка с автомобилем', 'сделка с подержанным автомобилем' | 'автомобиль' | 'автомобиль' |
|  |  |  |  |
| 3 | 'на проведение свадьбы', 'свадьба', 'сыграть свадьбу' | 'свадьба' | 'свадьба' |
|  |  |  |  |
| 4 | 'жилье', 'операции с жильем', 'покупка жилой недвижимости', 'покупка жилья', 'покупка жилья для сдачи', 'покупка жилья для семьи', 'покупка своего жилья', 'ремонт жилью', 'строительство жилой недвижимости' | 'жилье', 'жилой' | 'жилье' |
|  |  |  |  |
| 5 | 'недвижимость', 'операции с коммерческой недвижимостью', 'операции с недвижимостью', 'операции со своей недвижимостью', 'покупка коммерческой недвижимости', 'покупка недвижимости', 'строительство недвижимости', 'строительство собственной недвижимости' | 'недвижимость' | 'недвижимость' |

In [46]:
# функция для замены неявных дубликатов в поле purpose с использованием лемм
# если перечисленные леммы отсутствуют в поле purpose, функция вернет значение 'иная цель'
def purpose_category(purpose):
    try:
        if 'образование' in purpose:
            return 'образование'
        elif 'автомобиль' in purpose:
            return 'автомобиль'
        elif 'свадьба' in purpose:
            return 'свадьба'
        elif 'жилье' in purpose:
            return 'жилье' 
        elif 'жилой' in purpose:
            return 'жилье'
        elif 'недвижимость' in purpose:
            return 'недвижимость'
    except:
        return 'иная цель'

In [47]:
# замена неявных дубликатов в поле purpose
data['purpose'] = data['purpose'].transform(lambda x: purpose_category(x))

In [48]:
# проверка на отсутствие неявных дубликатов в поле purpose
sorted(data.purpose.unique())

['автомобиль', 'жилье', 'недвижимость', 'образование', 'свадьба']

**Вывод**

С помощью лемматизации устранены неявные дубликаты в поле `purpose`

### Категоризация данных

In [49]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,жилье,23
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,11
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,жилье,15
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,11
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,38


In [50]:
# словарь для поля education (уровень образования клиента)
education_dict = data[['education_id', 'education']].drop_duplicates().reset_index(drop=True)
education_dict

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [51]:
# словарь для поля family_status (семейное положение)
family_status_dict = data[['family_status_id', 'family_status']].drop_duplicates().reset_index(drop=True)
family_status_dict

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [52]:
# словарь для нового поля purpose_id (идентификатор  цели получения кредита)
columns = ['purpose_id', 'purpose']
rows = [[0,'жилье'], [1, 'автомобиль'], [2, 'образование'],
        [3, 'свадьба'], [4, 'недвижимость'], [5, 'иная цель']]

purpose_dict = pd.DataFrame(data=rows, columns=columns)
purpose_dict

,purpose_id,purpose
0,0,жилье
1,1,автомобиль
2,2,образование
3,3,свадьба
4,4,недвижимость
5,5,иная цель


In [53]:
# функция определения purpose_id
def purpose_category(purpose):
    purpose_list = list(purpose_dict['purpose'])
    for i in range(len(purpose_list)):
        if purpose == purpose_list[i]:
            return i

Добавим в датасет колонку purpose_id и заполним ее значением, рассчитанным с помощью функции.

In [54]:
data['purpose_id'] = data['purpose'].transform(lambda x: purpose_category(x))

In [55]:
# удаление столбцов education, family_status, purpose, для которых созданы словари
data = data.drop(['education', 'family_status', 'purpose'], axis = 1)

In [56]:
data.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,years_employed,purpose_id
0,1,8437,42,0,0,F,сотрудник,0,253875,23,0
1,1,4024,36,1,0,F,сотрудник,0,112080,11,1
2,0,5623,33,1,0,M,сотрудник,0,145885,15,0
3,3,4124,32,1,0,M,сотрудник,0,267628,11,2
4,0,14177,53,1,1,F,пенсионер,0,158616,38,3


In [57]:
# словарь для нового поля total_income_id (идентификатор ежемесячного дохода)
columns = ['total_income_id', 'total_income']
rows = [[0,'менее 100 тыс.'], [1, '100-200 тыс.'], [2, '200 тыс. и более']]

total_income_dict = pd.DataFrame(data=rows, columns=columns)
total_income_dict

,total_income_id,total_income
0,0,менее 100 тыс.
1,1,100-200 тыс.
2,2,200 тыс. и более


In [58]:
# функция определения категории по уровню дохода 
def total_income_category(total_income):
    if total_income < 100000:
        return 0
    elif 100000 <= total_income < 200000:
        return 1
    elif total_income >= 200000:
        return 2

Добавим в датасет колонку `total_income_id` и заполним ее значением категории по уровню дохода, рассчитанной с помощью функции.

In [59]:
data['total_income_id'] = data['total_income'].transform(lambda x: total_income_category(x))

In [60]:
data.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,years_employed,purpose_id,total_income_id
0,1,8437,42,0,0,F,сотрудник,0,253875,23,0,2
1,1,4024,36,1,0,F,сотрудник,0,112080,11,1,1
2,0,5623,33,1,0,M,сотрудник,0,145885,15,0,1
3,3,4124,32,1,0,M,сотрудник,0,267628,11,2,2
4,0,14177,53,1,1,F,пенсионер,0,158616,38,3,1


**Вывод**

Созданы словари для полей `education`, `family_status`, `purpose`, сами поля удалены из датасета, что уменьшило размер датасета и соответственно увеличило скорость работы с ним.

В датасет добавлено новое поле `total_income_id` (идентификатор ежемесячного дохода)  и создан соответствующий словарь. Данная информация будет использоваться в дальнейшем при исследовании.

## Шаг 3. Ответьте на вопросы

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

Сгруппируем данные по полю `children` (количество детей в семье) и посчитаем значения `debt` (имел ли задолженность по возврату кредитов):
- count - количество клиентов в группе
- sum - число должников
- mean - доля должников в каждой группе

In [61]:
debt_children = data.groupby('children').agg({'debt': ['count', 'sum', 'mean']})
debt_children

debt                
          count   sum      mean
children                       
0         14154  1064  0.075173
1          4809   444  0.092327
2          2128   202  0.094925
3           330    27  0.081818
4            41     4  0.097561
5             9     0  0.000000

**Выводы:**
- Клиенты без детей берут кредиты чаще, чем клиенты с детьми.
- Самая низкая доля должников в группе клиентов без детей.
- Самая высокая доля должников в группе с четырьмя детьми.
- Доли должников в группах с одним ребенком и двумя детьми отличаются незначительно.
- В группе с пятью детьми должников нет.

В группах с тремя, четырьмя и пятью детьми на порядок меньше клиентов, закономерность в доле должников не прослеживается. эти три группы можно объединить в одну - **многодетная семья**. 

Сгруппируем данные по количеству детей в три группы:
- семья без детей
- семья с 1 или 2 детьми
- многодетная семья

In [62]:
# словарь для нового поля children_category_id (идентификатор по количеству детей)
columns = ['children_category_id', 'children_category']
rows = [[0,'семья без детей'], [1, 'семья с 1 или 2 детьми'], [2, 'многодетная семья']]

children_category_dict = pd.DataFrame(data=rows, columns=columns)
children_category_dict

,children_category_id,children_category
0,0,семья без детей
1,1,семья с 1 или 2 детьми
2,2,многодетная семья


In [63]:
# функция для определения группы по количеству детей
def children_category(children):
    if children == 0:
        return 0
    elif 0 < children < 3:
        return 1
    elif children > 2:
        return 2

In [64]:
# добавим в датасет колонку purpose_id и заполним ее значением, рассчитанным с помощью функции
data['children_category_id'] = data['children'].transform(lambda x: children_category(x))

Еще раз сгруппируем данные, только теперь по полю `children_category_id`. Добавим к таблице значения словаря children_category_dict методом merge() и удалим колонку `children_category_id` методом drop()

In [65]:
children_debt = data.groupby('children_category_id').agg({'debt': ['count', 'sum', 'mean']})\
        .merge(children_category_dict, on='children_category_id')\
        .drop(['children_category_id'], axis = 1)
children_debt

C:\Users\79853\miniconda3\lib\site-packages\pandas\core\frame.py:9190: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left,1 on the right)
  return merge(


,"(debt, count)","(debt, sum)","(debt, mean)",children_category
0,14154,1064,0.075173,семья без детей
1,6937,646,0.093124,семья с 1 или 2 детьми
2,380,31,0.081579,многодетная семья


In [66]:
# переименуем колонки в сводной таблице
children_debt.columns = ['всего клиентов', 'должники', 'доля должников, %', 'категория семьи по количеству детей']

In [67]:
# переведем долю должников в проценты
children_debt['доля должников, %'] = round(children_debt['доля должников, %'] * 100, 1)

In [68]:
children_debt

,всего клиентов,должники,"доля должников, %",категория семьи по количеству детей
0,14154,1064,7.5,семья без детей
1,6937,646,9.3,семья с 1 или 2 детьми
2,380,31,8.2,многодетная семья


**Вывод**

Согласно результатам исследования среди клиентов без детей самый низкий процент должников - 7.5%, самый высокий процент должников среди клиентов с одним или двумя детьми - 9.3%. У многодетных семей средний показатель - 8.2% должников.

Можно сделать вывод, что **клиенты без детей чаще возвращают кредиты в срок**.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Сгруппируем данные по полю `family_status_id` (идентификатор семейного положения) и посчитаем значения `debt` (имел ли задолженность по возврату кредитов):
- count - количество клиентов в группе
- sum - число должников
- mean - доля должников в каждой группе

Добавим к таблице значения словаря family_status_dict методом merge() и удалим колонку `family_status_id` методом drop()

In [69]:
debt_family_status = data.groupby('family_status_id').agg({'debt': ['count', 'sum', 'mean']})\
        .merge(family_status_dict, on='family_status_id')
debt_family_status

C:\Users\79853\miniconda3\lib\site-packages\pandas\core\frame.py:9190: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left,1 on the right)
  return merge(


,family_status_id,"(debt, count)","(debt, sum)","(debt, mean)",family_status
0,0,12344,931,0.075421,женат / замужем
1,1,4163,388,0.093202,гражданский брак
2,2,959,63,0.065693,вдовец / вдова
3,3,1195,85,0.071130,в разводе
4,4,2810,274,0.097509,Не женат / не замужем


In [70]:
# переименуем колонки в сводной таблице, кроме колонки family_status_id
debt_family_status.columns = ['family_status_id', 'всего клиентов', 'должники', 'доля должников, %', 'семейное положение']

In [71]:
# переведем долю должников в проценты
debt_family_status['доля должников, %'] = round(debt_family_status['доля должников, %'] * 100, 1)

In [72]:
debt_family_status

,family_status_id,всего клиентов,должники,"доля должников, %",семейное положение
0,0,12344,931,7.5,женат / замужем
1,1,4163,388,9.3,гражданский брак
2,2,959,63,6.6,вдовец / вдова
3,3,1195,85,7.1,в разводе
4,4,2810,274,9.8,Не женат / не замужем


**Вывод**

- Женатые / замужние клиенты берут кредиты чаще, чем клиенты с другим семейным положением.
- Наименьший процент должников среди овдовевших клиентов, - 6.6%.
- Наибольший процент должников среди не женатых / не замужних клиентов, - 9.8%. 

Скорее всего, зависимость в данном случае обусловлена не семейным положением, а скорее возрастом клиентов, т.к. чем старше человек, тем он становится ответственнее и старается не допускать задолжностей по своим кредитам. Ниже рассчитаем медианный возраст клиетов во всех группах по семейному положению, чтобы подтвердить или опровергнуть эту гипотезу.

Сгруппируем данные по полю `family_status_id` (идентификатор семейного положения) и посчитаем **средний возраст клиетов** в каждой группе.

In [73]:
family_status_dob_years = data.groupby('family_status_id')['dob_years'].mean().astype('int').reset_index()
family_status_dob_years

,family_status_id,dob_years
0,0,43
1,1,42
2,2,56
3,3,45
4,4,38


Добавим к таблице с результатами колонку со средним возрастом клиетов, удалим колонку `family_status_id`, переименуем колонку `dob_years` в `средний возраст`.

In [74]:
debt_family_status = debt_family_status.merge(family_status_dob_years, on='family_status_id')\
        .drop(['family_status_id'], axis = 1)\
        .rename(columns={"dob_years": "средний возраст"})
debt_family_status

,всего клиентов,должники,"доля должников, %",семейное положение,средний возраст
0,12344,931,7.5,женат / замужем,43
1,4163,388,9.3,гражданский брак,42
2,959,63,6.6,вдовец / вдова,56
3,1195,85,7.1,в разводе,45
4,2810,274,9.8,Не женат / не замужем,38


Гипотеза с возрастом клиетов подтвердилась, наименьшая доля задолжностей 6.6% среди самой старшей группы 56 лет (семейное положение - вдовец / вдова). наибольшая доля задолжностей 9.8% среди самой молодой группы 38 лет (семейное положение - Не женат / не замужем).

Можно сделать вывод, что **существует косвенная зависимость между семейным положением и возвратом кредитов в срок**, но скорее всего это связано с возрастом клиентов: чем старше клиент, тем выше ответственность, тем меньше задолжностей по кредитам.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Составим сводную таблицу методом **pivot_table()**: для параметра index возьмем переменную `total_income_id`, для параметра values - переменную `debt`. В качестве функций aggfunc:

- count - количество клиентов в группе
- sum - число должников
- mean - доля должников в каждой группе

Добавим к таблице значения словаря total_income_dict методом merge() и удалим колонку `total_income_id` методом drop()

In [75]:
total_income_debt = pd.pivot_table(data, index='total_income_id', columns=None, values='debt',
                                   aggfunc=['count', 'sum', 'mean'])\
                      .merge(total_income_dict, on='total_income_id')\
                      .drop(['total_income_id'], axis = 1)
total_income_debt

C:\Users\79853\miniconda3\lib\site-packages\pandas\core\frame.py:9190: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left,1 on the right)
  return merge(


,"(count, debt)","(sum, debt)","(mean, debt)",total_income
0,4463,354,0.079319,менее 100 тыс.
1,11941,1029,0.086174,100-200 тыс.
2,5067,358,0.070653,200 тыс. и более


In [76]:
# переименуем колонки в сводной таблице
total_income_debt.columns = ['всего клиентов', 'должники', 'доля должников, %', 'ежемесячный доход']

In [77]:
# переведем долю должников в проценты
total_income_debt['доля должников, %'] = round(total_income_debt['доля должников, %'] * 100, 1)

In [78]:
total_income_debt

,всего клиентов,должники,"доля должников, %",ежемесячный доход
0,4463,354,7.9,менее 100 тыс.
1,11941,1029,8.6,100-200 тыс.
2,5067,358,7.1,200 тыс. и более


**Вывод**

Клиенты c ежемесячным доходом 100-200 тыс. берут кредиты чаще, чем клиенты с другим доходом.

Наибольший процент задолжностей среди клиентов с ежемесячным доходом 100-200 тыс., - 8.6%.

Можно сделать вывод, что **уровень дохода влияет на возврат кредита в срок**. Так клиенты со средним уровнем дохода 100-200 тыс. в месяц имеют более высокий процент задолжностей по кредитам, с уменьшением и увеличением уровня доходов процент задолжностей становится ниже.

### Как разные цели кредита влияют на его возврат в срок?

Составим сводную таблицу методом **pivot_table()**: для параметра index возьмем переменную `purpose_id`, для параметра values - переменную `debt`. В качестве функций aggfunc:

- count - количество клиентов в группе
- sum - число должников
- mean - доля должников в каждой группе

Добавим к таблице значения словаря purpose_dict методом merge() и удалим колонку `purpose_id` методом drop()

In [79]:
debt_purpose = pd.pivot_table(data, index='purpose_id', columns=None, values='debt', aggfunc=['count', 'sum', 'mean'])\
        .merge(purpose_dict, on='purpose_id')\
        .drop(['purpose_id'], axis = 1)
debt_purpose

C:\Users\79853\miniconda3\lib\site-packages\pandas\core\frame.py:9190: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left,1 on the right)
  return merge(


,"(count, debt)","(sum, debt)","(mean, debt)",purpose
0,5692,397,0.069747,жилье
1,4308,403,0.093547,автомобиль
2,4014,370,0.092177,образование
3,2335,186,0.079657,свадьба
4,5122,385,0.075166,недвижимость


In [80]:
# переименуем колонки в сводной таблице
debt_purpose.columns = ['всего клиентов', 'должники', 'доля должников, %', 'цель кредита']

In [81]:
# переведем долю должников в проценты
debt_purpose['доля должников, %'] = round(debt_purpose['доля должников, %'] * 100, 1)

In [82]:
debt_purpose

,всего клиентов,должники,"доля должников, %",цель кредита
0,5692,397,7.0,жилье
1,4308,403,9.4,автомобиль
2,4014,370,9.2,образование
3,2335,186,8.0,свадьба
4,5122,385,7.5,недвижимость


**Вывод**

Большее количество кредитов берут на жилье, на втором месте - недвижимость (в том числе коммерческая).

На жилье и недвижимость задолжности по кредитам ниже, чем на другие цели. Это может быть связано с тем, что клиенты более ответственно относятся к выплате кредитов на более важные для них цели. т.е. чем важнее цель, тем более ответственное отношение у клиента к выплате кредита.

Также можно предположить, что это связано с тем, что стоимость жилья и недвижимости на порядок выше других пунктов, поэтому клиенты более ответственно относятся к таким кредитам, т.к. задолжность может грозить очень высокими процентами переплаты.

Можно сделать вывод, что **цель кредита влияет на его возврат в срок**.

## Шаг 4. Общий вывод

Клиенты без детей берут и возвращают кредиты чаще, чем клиенты с детьми.

Женатые/замужние клиенты берут кредиты чаще, чем клиенты с другим семейным положением.

Существует косвенная зависимость между семейным положением и возвратом кредитов в срок, которую скорее можно связать с возрастом клиентов: чем старше клиент, тем выше ответственность, тем меньше задолжностей по кредитам.

Уровень дохода также влияет на возврат кредита в срок. Так, клиенты со средним уровнем дохода 100-200 тыс. в месяц имеют более высокий процент задолжностей по кредитам, с уменьшением и увеличением уровня доходов клиентов процент задолжностей становится ниже.

Больше всего кредитов берут на жилье и недвижимость (в том числе коммерческую). И чем важнее цель для клиента, тем более ответственное отношение у него к выплате кредита.